<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Plotly_Testing_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#@title Dependencies (double click to review code)
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
!pip install python-exchangeratesapi

context = ssl._create_unverified_context()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}


def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records

def get_data_from_MAS(url):
    # Balance sheet
    url = url
    req = urllib.request.Request(url,headers=headers)
    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        path = url+ "&offset=" + str(offset)
        path_header = urllib.request.Request(path,headers=headers)
        with urllib.request.urlopen(path_header) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['offset'] <= int(data['result']['total']):
            offset += 100
        else:
            break

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records

print("dependencies ready")


dependencies ready


In [14]:
#@title # **Monetary Base**(double click to review code)
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

type(get_data_from_HKMA(MBC))
#Getting data from  7.21(Monetary Base – End of period figures) and store it as a pandas dataframe type
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

# cert_of_indebt - Certificates of Indebtedness

# gov_notes_coins_circulation - Government notes/coins in circulation

# aggr_balance_bf_disc_win - 	Aggregate Balance before Discount Window
# outstanding_efbn - Outstanding Exchange Fund Bills and Notes

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn'],var_name='Size of the Monetary Base', value_name='HK$ million' )

#Plot
fig = px.line(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')

#Plot Format
fig.update_layout(title_text="Monetary Base – End of period figures")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()


A part of the monetary liabilities of a central bank. The Monetary Base is defined, at the minimum, as the sum of the currency in circulation (banknotes and coins) and the balance of the banking system held with the central bank (the reserve balance or the clearing balance). In Hong Kong, the Monetary Base comprises Certificates of Indebtedness (for backing the banknotes issued by the note-issuing banks), government-issued currency in circulation, the balance of the clearing accounts of banks kept with the HKMA (the Aggregate Balance), and Exchange Fund Bills and Notes.


In [15]:
#@title #1. Money Multiplier = M3 / MB (HKD)  (double click to review code)
MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.

#Getting data from 2.21 (Money supply – Adjusted for foreign currency swap deposits) and store it as a pandas dataframe type
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))

#m3_hkd- M3 HK$(Adjusted to include foreign currency swap deposits)

#Since the duration of m3_hkd and mb_bf_disc_win_total are not the same

#We merge them as a new dataset by the end_of_month

m3_hkd = msa_df[['end_of_month','m3_hkd']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_hkd'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])

#Plot
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb", title='M3(hkd) / MB(Total)')

#Plot Format
# Set x-axis title
fig.update_xaxes(title_text="end_of_month")
# Set y-axis title
fig.update_yaxes(title_text="M3 / MB")

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()




The total stock of money available in the economy. Hong Kong has three measures of money supply:

Money Supply definition 1 (M1): The sum of legal tender notes and coins held by the public plus customers' demand deposits placed with banks.

Money Supply definition 2 (M2): M1 plus customers' savings and time deposits with banks plus negotiable certificates of deposit (NCDs) issued by banks held outside the banking sector.

Money Supply definition 3 (M3): M2 plus customers' deposits with restricted licence banks and deposit-taking companies plus NCDs issued by these institutions held outside the banking sector.

Among these three series, HK M1 exhibits a significant seasonal pattern, whereas there is no strong evidence of seasonality in broad money (HK M2 and HK M3). Seasonally adjusted series of HK M1 and its components (i.e. cash held by the public and demand deposits) are compiled and published by the HKMA.

In [16]:
#@title #2. Total deposits & M3 (double click to review code)
TD = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/customer-deposits-by-currency?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

#Getting data from 3.2 (Customer deposits by currency) and store it as a pandas dataframe type
td_records_df = pd.json_normalize(get_data_from_HKMA(TD))

#deposits_all - Deposits with authorized institutions (Adjusted for foreign currency swap deposits) All currencies

fig = go.Figure()

fig.add_trace(go.Scatter(x=msa_df["end_of_month"], y = msa_df[['end_of_month','m3_total']]
["m3_total"], mode='lines',
              name ='M3(Total)'))

fig.add_trace(go.Scatter(x=td_records_df["end_of_month"], y =td_records_df["deposits_all"], mode='lines',
              name ='Deposits(Total)'))

fig.update_layout(title_text="Total deposits & M3")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()

In [17]:
#@title #3.1 Loan to Deposits (double click to review code)
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

#Getting data from 3.9.1 (Balance sheet – Authorized institutions) and store it as a pandas dataframe type
bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))

fig = make_subplots(rows=1, cols=3)

#Calculate the ratio

#Assets Loans and advances to customers HK$ - asst_loanadvtocust_hkd
#Liabilities Deposits from customers HK$  - asst_loanadvtocust_hkd
bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])

#Assets Loans and advances to customers F.C. - asst_loanadvtocust_fc
#Liabilities Deposits from customers F.C.  - liab_depcust_fc
bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])

#Assets Loans and advances to customers Total - asst_loanadvtocust_total
#Liabilities Deposits from customers Total  - liab_depcust_total
bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])


#Plot
fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_hkd"], mode='lines',
              name ='Loan to Deposits(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_fc"], mode='lines',
              name ='Loan to Deposits(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_total"], mode='lines',
              name ='Loan to Deposits (Total))'), row = 1, col=3)

#Plot Format
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="Loan to Deposits")
fig.update_layout(title_text="Loan to Deposits")

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)


fig.show()

In [18]:
#@title #3.2 Loan and Deposits (double click to review code)

#Plot
fig = make_subplots(rows=1, cols=3)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_hkd"], mode='lines',
              name ='Loan(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_hkd"], mode='lines',
              name ='Deposits(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_fc"], mode='lines',
              name ='Loan(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_fc"], mode='lines',
              name ='Deposits(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_total"], mode='lines',
              name ='Loan(Total))'), row = 1, col=3)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_total"], mode='lines',
              name ='Deposits (Total))'), row = 1, col=3)

#Plot Format
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="HK$ million")
fig.update_layout(title_text="Loan and Deposits")

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)


fig.show()

In [19]:
#@title #4.Centaline index (double click to review code)
!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv

#Getting data from BruceAUHK GitHub(Hv to manually update the lastest CCI data monthly) and store it as a pandas dataframe type

cci = pd.read_csv('cci.csv')

#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=cci["date"], y = cci["cci"], mode='lines', name="Centaline index"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   


#Plot format

# Add figure title
fig.update_layout( xaxis_range=['1997-7','2020-06'],
    title_text="CCI VS MB"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>CCI</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()


--2020-07-30 07:08:17--  https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4205 (4.1K) [text/plain]
Saving to: ‘cci.csv.1’

cci.csv.1           100%[===================>]   4.11K  --.-KB/s    in 0s      

2020-07-30 07:08:17 (59.2 MB/s) - ‘cci.csv.1’ saved [4205/4205]



Announced every month, latest on 2020/ 07/ 18, reflecting secondary private residential property price for 2020/05 (based on formal sale & purchase date; on average, formal S&P are signed within 14 days after preliminary S&P)

In [20]:
#@title #5.1 MB VS HIBOR(double click to review code)
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'

#Getting data from 6.3.1 (Hong Kong Interbank Interest Rates – End of period figures) and store it as a pandas dataframe type
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))

#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   

#Plot format

# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="MB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")


# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()

 


In [21]:
#@title ##5.2 AB VS HIBOR(double click to review code)
#Plot 
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['aggr_balance_bf_disc_win'], mode='lines', name="AB"),secondary_y=True,)   

#Plot format
# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="AB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.show()

 


In [22]:
#@title #6 FPS Transactions and Registrations(double click to review code)
fps_val_hkd = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-val?sortby=end_of_month&sortorder=asc'
#Getting data from 3.12.4 (Clearing House Statistics – Turnover of HKD FPS payment (Value)) and store it as a pandas dataframe type
fps_val_hkd_df = pd.json_normalize(get_data_from_HKMA(fps_val_hkd))

fps_reg = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-fps-reg?sortby=end_of_month&sortorder=asc'
#Getting data from 3.12.2	(Faster Payment System Addressing Service Registrations) and store it as a pandas dataframe type
fps_reg_df = pd.json_normalize(get_data_from_HKMA(fps_reg))

#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#total - Real time credit transfer payment Total real time credit transfer payment + Real time direct debit payment + Batch mode payment
fig.add_trace(go.Bar(x=fps_val_hkd_df['end_of_month'], y = fps_val_hkd_df['total'], name="FPS Turnover of HKD FPS payment (Value in HK$ thousand)"),secondary_y=False)   

#total_grand_total - Total Grand-total
fig.add_trace(go.Scatter(x=fps_reg_df['end_of_month'], y = fps_reg_df['total_grand_total'], mode='lines+markers', name="FPS Addressing Service Registrations Total Grand-total"),secondary_y=True,)  

#Plot format
# Add figure title
fig.update_layout( xaxis_range=['2018-9','2020-06'],
    title_text="Transactions and Registrations"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>FPS Turnover of HKD FPS payment (Value in HK$ thousand)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>FPS Addressing Service Registrations Total Grand-total</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)


fig.show()

 


In [23]:
#@title #7 MB & GDP (Developping) (double click to review code)

econ= 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/financial/economic-statistics?sortby=end_of_month&sortorder=asc'
#Getting data from 1.4	Economic statistics and store it as a pandas dataframe type
econ_df = pd.json_normalize(get_data_from_HKMA(econ))


#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#total - Real time credit transfer payment Total real time credit transfer payment + Real time direct debit payment + Batch mode payment
fig.add_trace(go.Bar(x=econ_df['end_of_month'], y = econ_df['real_gdp'], name="Real GDP1 (at chained 2016 dollars)"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   


#Plot format
# Add figure title
fig.update_layout(  xaxis_range=['2019-6','2020-06'], title_text="MB & GDP")

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>FPS Turnover of HKD FPS payment (Value in HK$ thousand)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>FPS Addressing Service Registrations Total Grand-total</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([  
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)


fig.show()

 


In [24]:
#@title #8 HK VS SGP Equity market Total stock market capitalization (Developping) (double click to review code)

capital_hk= 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/financial/capital-market-statistics?sortby=end_of_month&sortorder=asc'
#Getting data from 1.3	Capital market statistics and store it as a pandas dataframe type
capital_hk_df = pd.json_normalize(get_data_from_HKMA(capital_hk))


capital_sp = 'https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=1c1713de-6b5e-475d-bc1e-b6a45b3e063e&limit=100&fields=end_of_month,mkt_cap_total&sort=end_of_month%20asc'
#Getting data from Table III.7 SGX-ST: Price Index, Number of Listed Companies, Turnover and Capitalisation, Monthly
capital_sp_df = pd.json_normalize( get_data_from_MAS(capital_sp))


from exchangeratesapi import Api
api = Api()
# Get the latest foreign exchange rates:
api.get_rates()

capital_sp_df['mkt_cap_total_hkd'] = capital_sp_df["mkt_cap_total"].astype(float).multiply(api.get_rate('SGD', 'HKD'))


#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#total - Real time credit transfer payment Total real time credit transfer payment + Real time direct debit payment + Batch mode payment
#fig.add_trace(go.Bar(x=econ_df['end_of_month'], y = econ_df['real_gdp'], name="Real GDP1 (at chained 2016 dollars)"),secondary_y=False)   

fig.add_trace(go.Scatter(x=capital_hk_df['end_of_month'], y = capital_hk_df['eq_mkt_ttl_stock_cap'], mode='lines', name="Hong Kong Equity market Total stock market capitalization"),secondary_y=False)   
fig.add_trace(go.Scatter(x=capital_sp_df['end_of_month'], y = capital_sp_df['mkt_cap_total_hkd'], mode='lines', name="Singapore Equity market Total stock market capitalization"),secondary_y=False)   


#Plot format
# Add figure title
fig.update_layout(title_text="HK VS SGP Equity market Total stock market capitalization")

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>(HK$ million)</b>", secondary_y=False)
# Add figure title
fig.update_layout( xaxis_range=['2019-4','2020-06']
)


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"), 
          dict(count=6, label="6Months", step="month", stepmode="backward"), 
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)


fig.show()

 





In [39]:
#@title #9 Liabilities due to other authorized institutions(Developping) (double click to review code)

loans= 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money-markets/liab-dt-other-ais?sortby=end_of_month&sortorder=asc'
#Getting data from 1.3	Capital market statistics and store it as a pandas dataframe type
loans_df = pd.json_normalize(get_data_from_HKMA(loans))

loans_df['total_loans_hkd_pec'] = loans_df["total_loans_hkd"].divide(loans_df["total_loans"])*100 
loans_df['total_loans_fc_pec'] = loans_df["total_loans_fc"].divide(loans_df["total_loans"])*100 

#Plot
#Convert dataframe to dataframe long format
loans_dfl = pd.melt(loans_df, id_vars = ['end_of_month'], value_vars =['total_loans_hkd_pec', 'total_loans_fc_pec' ],var_name='total_loans', value_name='Percentages' )

#Plot
fig = px.area(loans_dfl, x='end_of_month' , y='Percentages', color = 'total_loans')

#Plot Format

#Plot format
# Add figure title
fig.update_layout(title_text="Liabilities due to other authorized institutions")

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Percentages/b>", secondary_y=False)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"), 
          dict(count=6, label="6Months", step="month", stepmode="backward"),
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=5, label="5y", step="year", stepmode="backward"), 
          dict(step="all")
      ])
    )
)


fig.show()

 



